In [7]:
##############################################
# Import Library
##############################################

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from scipy.stats import bernoulli, zscore
from tqdm.notebook import tqdm
import time

import calibration_metric
import importlib
importlib.reload(calibration_metric)
from calibration_metric import ece, ace, tce

In [ ]:
##############################################
# Load Model Outputs
##############################################

labels = np.load("./Dataset/DL/labels.npy")
preds_00 = np.load("./Dataset/DL/preds_00.npy")
preds_01 = np.load("./Dataset/DL/preds_01.npy")
preds_02 = np.load("./Dataset/DL/preds_02.npy")
preds_03 = np.load("./Dataset/DL/preds_03.npy")
preds_04 = np.load("./Dataset/DL/preds_04.npy")
preds = [ preds_00, preds_01, preds_02, preds_03, preds_04 ]
names = ['alexnet', 'vgg19', 'resnet18', 'resnet50', 'resnet152']


In [8]:
##############################################
# Measure Errors
##############################################

def do_experiment(labels, preds, names):
    tces = np.zeros((1, len(preds)))
    taes = np.zeros((1, len(preds)))
    eces = np.zeros((1, len(preds)))
    aces = np.zeros((1, len(preds)))
    mces = np.zeros((1, len(preds)))
    maes = np.zeros((1, len(preds)))
    times = np.zeros((6, len(preds)))
    
    for ith, preds_ith in enumerate(preds):
        print("- Model: " + names[ith])
        
        filename = "./Plot/DL/" + names[ith] 
        n_min = int( len(labels) / 20 )
        n_max = int( len(labels) / 5 )
        n_bin = 10
        preds_ith[preds_ith >= 1.0] = 1.0
        
        start = time.time()
        tces[0, ith] = tce(preds_ith, labels, n_min=n_min, n_max=n_max, savepath=filename+"_tce.png")
        end = time.time()
        times[0, ith] = end - start
        
        start = time.time()
        taes[0, ith] = tce(preds_ith, labels, strategy="quantile", n_bin=n_bin)
        end = time.time()
        times[1, ith] = end - start
        
        start = time.time()
        eces[0, ith] = ece(preds_ith, labels, n_bin=n_bin, mode='l1', savepath=filename+"_ece.png")
        end = time.time()
        times[2, ith] = end - start
        
        start = time.time()
        aces[0, ith] = ace(preds_ith, labels, n_bin=n_bin, mode='l1', savepath=filename+"_ace.png")
        end = time.time()
        times[3, ith] = end - start
        
        start = time.time()
        mces[0, ith] = ece(preds_ith, labels, n_bin=n_bin, mode='inf')
        end = time.time()
        times[4, ith] = end - start
        
        start = time.time()
        maes[0, ith] = ace(preds_ith, labels, n_bin=n_bin, mode='inf')
        end = time.time()
        times[5, ith] = end - start
        
    result = np.c_[ tces[0, :], taes[0, :], eces[0, :], aces[0, :], mces[0, :], maes[0, :] ]
    result = np.r_[ result, times.mean(axis=1).reshape(1, -1) ]
    result = pd.DataFrame(result)
    result.index = names + ["time"]
    result.columns = ['TCE', 'TCE(Q)', 'ECE', 'ACE', 'MCE', 'MCE(Q)']
    result.to_csv("./Result/DL/summary.csv", float_format="%.4f")
    pd.options.display.float_format = '{:.4f}'.format
    display(result)

do_experiment(labels, preds, names)



- Model: alexnet
- Model: vgg19
- Model: resnet18
- Model: resnet50
- Model: resnet152
- Completed


,TCE,TCE(Q),ECE,ACE,MCE,MCE(Q)
alexnet,42.7360,43.7920,0.0070,0.0070,0.1496,0.0528
vgg19,23.5660,22.8880,0.0028,0.0028,0.2148,0.0247
resnet18,29.9340,31.7780,0.0042,0.0042,0.2368,0.0350
resnet50,24.5960,23.0540,0.0020,0.0018,0.1911,0.0152
resnet152,16.0860,22.1600,0.0012,0.0013,0.1882,0.0101
time,71.7837,63.6940,0.4873,0.4221,0.0046,0.0063
